Recently, I was inspired to create an arbitrage sports betting model to see how often these opprtunities emerge. Before incorporating actual sports books, developping an accurate model was necessary. Intially, the model was quickly developped in Excel VBA using excel solver, however, the model might require too many iterations along with too many constraints to operate efficently, so it was converted to Python.

In [4]:
import scipy.optimize as opt

This function is equivalent to inputting the betting odds and total betting amount within an input box in VBA, and running excel solver to produce the optimal result.

The only difference is that within this python library, the goal of the objective function needs to be to minimize it. The reason for the selected objective function was due to additional research. Within an arbitrage bet, the optimal output happens when the total money won from bet 1 and bet 2 are equal. The difference of squares was taken to ensure the answer stays positive, and mathematically it is typically better to square something as opposed to using the absolute difference.

In [5]:
def run_solver():
    # Prompt the user for input (equivalent to Excel's InputBox)
    input1 = input("Please enter if the first bet is on the Favourite or Underdog (Need to type either F or U): ")
    
    input2 = int(input("Enter the odds of the first bet (Just the Number): "))
    
    input3 = input("Please enter if the second bet is on the Favourite or Underdog (Need to type either F or U): ")
    
    input4 = int(input("Enter the odds of the second bet (Just the Number): "))
    
    input5 = float(input("Enter the maximum willing to bet: "))
    
    # Display the input values
    print(f"You entered: {input1} and {input2} for bet 1 and {input3} and {input4} for bet 2")
    
    # Define the objective function (for example, minimizing something based on the inputs)
    # Example: Minimize a function that depends on the inputs (you can define your own function)
    
    # This could represent some mathematical objective based on the user's input
    def objective(x):
        # Just an example: bet amounts as the decision variables
        bet1 = x[0]
        bet2 = x[1]

        if input1 == 'F':
            total_money_1 = (bet1 * 100 / input2) + bet1
        
        elif input1 == 'U':
            total_money_1 = (bet1 * input2 / 100) + bet1
        
        else:
            raise ValueError("Invalid input1 parameter. Run the model and try again.")

        if input3 == 'F':
            total_money_2 = (bet2 * 100 / input4) + bet2
        
        elif input3 == 'U':
            total_money_2 = (bet2 * input4 / 100) + bet2
        
        else:
            raise ValueError("Invalid input3 parameter. Run the model and try again.")
        
        return (total_money_1 - total_money_2) ** 2  # Minimize the difference for equal payout

    # Constraints: 
    constraints = (
        {"type": "ineq", "fun": lambda x: input5 - (x[0] + x[1])},  # Sum of bet1 and bet2 equals input5
        {"type": "ineq", "fun": lambda x: x[0]},  # bet1 >= 0
        {"type": "ineq", "fun": lambda x: x[1]},  # bet2 >= 0
        {"type": "ineq", "fun": lambda x: input5 - x[0]},  # bet1 <= input5
        {"type": "ineq", "fun": lambda x: input5 - x[1]},  # bet2 <= input5
    )

    # Initial guess for the optimization (you can adjust the initial guess as needed)
    initial_guess = [input5 / 2, input5 / 2]  # Assuming you want to split input5 evenly between the two bets

    # Perform the optimization
    result = opt.minimize(objective, initial_guess, constraints=constraints, options={'maxiter': 1000000}, method='SLSQP')

    # Check if the optimization was successful
    if result.success:
        # Optimization succeeded, display the results
        optimized_bets = result.x
        print(f"Bet 1 Status: {input1} {input2}")
        print(f"Bet 2 Status: {input3} {input4}")
        print(f"Optimal Bet 1: {optimized_bets[0]:.2f}")
        print(f"Optimal Bet 2: {optimized_bets[1]:.2f}")

        # Calculate and print the value of the objective function
        if input1 == 'F':
            total_money_1 = (optimized_bets[0] * 100 / input2) + optimized_bets[0]
        else:
            total_money_1 = (optimized_bets[0] * input2 / 100) + optimized_bets[0]
        if input3 == 'F':
            total_money_2 = (optimized_bets[1] * 100 / input4) + optimized_bets[1]
        else:
            total_money_2 = (optimized_bets[1] * input4 / 100) + optimized_bets[1]
        
        profit = min(total_money_1, total_money_2) - input5
        print(f"Profit: ${profit:.2f}")
    else:
        print("Solver failed to find a solution.")

In [6]:
if __name__ == "__main__":
    run_solver()

You entered: U and 125 for bet 1 and U and 130 for bet 2
Bet 1 Status: U 125
Bet 2 Status: U 130
Optimal Bet 1: 101.09
Optimal Bet 2: 98.89
Profit: $27.45


Based on several attempts, the model is not converging on the proper value. This could be due to several factors, but this model will not be effective. There will be a second model made based on an article from Medium.